In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Sep 27 23:35:47 2020

@author: prana
"""
import cv2
import numpy as np


from scipy.signal import convolve2d
import matplotlib.pyplot as plt

import glob 
import os



In [2]:
def findCorners(img):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    corners = cv2.goodFeaturesToTrack(gray,128,0.01,10)
    #print(corners.shape)
    cv2.waitKey(0)
    corners = np.int0(corners)
    
    for i in corners:
        x,y = i.ravel()
        cv2.circle(img,(x,y),3,255,-1)
        
    return img,corners

def findFeatureVectors(corners1,img1):
    featureVector1=[]
    for c in corners1:
        #print(c)
        x,y=c[0]
        
        if y-20<=0:
            y=20
        if y>img1.shape[0]:
            y=img1.shape[0]-1-20
            
        if x-20<=0:
            x=20
        if x>img1.shape[1]:
            x=img1.shape[1]-1-20
        x=int(x)
        y=int(y)
        #print(x,y)
        block40x40=img1[y-20:y+20,x-20:x+20]
        #print(block40x40.shape)
        block40x40=cv2.resize(block40x40,(40,40))
        blur = cv2.GaussianBlur(block40x40,(5,5),0)
        feature8x8=cv2.resize(blur,(8,8))
        featureVector=np.reshape(feature8x8,(64,1))
        featureVector=featureVector-np.mean(featureVector)
        featureVector=featureVector/np.std(featureVector)
        #print(featureVector)
        featureVector1.append(featureVector)
    
    return featureVector1



def matchpair(featureVector1,featureVector2,bestcorners1,bestcorners2):
    featureVector1=np.array(featureVector1)
    featureVector2=np.array(featureVector2)
    fv1shape=featureVector1.shape
    fv2shape=featureVector2.shape
    matches=[]
    for i in range(0,fv1shape[0]):
        smallest_dist=[1000000000,0,0]
        second_smallest_dist=[100000000000,0,0]
        for j in range(0,fv2shape[0]):
            fv1=featureVector1[i]
            fv2=featureVector2[j]
            ssum=0
            for k in range(0,len(fv1)):
                ssum+=(fv1[k]-fv2[k])**2
            if ssum<smallest_dist[0]:
                smallest_dist=[ssum,i,j]
            elif ssum<second_smallest_dist[0]:
                second_smallest_dist=[ssum,i,j]
        
        if smallest_dist[0]>second_smallest_dist[0]:
            temp=deep_copy(second_smallest_dist)
            second_smallest_dist=smallest_dist
            smallest_dist=temp
        ratio=smallest_dist[0]/second_smallest_dist[0]
        if ratio<0.75:
            matches.append(smallest_dist)
    return matches
            
            
    
    

def get_val(r):
    return r[0]

def ANMS_(img,num):
    features=cv2.goodFeaturesToTrack(img,1000, 0.01,10)
    r=1000000000000*np.ones((features.shape[0],3))
    ED=0
    for i in range(features.shape[0]):
        for j in range(features.shape[0]):
            xi,yi=features[i][0]
            xj,yj=features[j][0]
           
            if img[int(yi)][int(xi)]>img[int(yj)][int(xj)]:
                ED=(xj-xi)**2 + (yj-yi)**2
            
            if ED<r[i][0]:
                r[i][0]=ED
                r[i][1]=xi
                r[i][2]=yi
    
    feat=r[np.argsort(-r[:,0])]             ########sort descending
    bestcorners=feat[:num,:]
    #r.sort( key=get_val ,reverse=True)
    #print(bestcorners[0])
    #print(bestcorners.shape)
    return bestcorners
    
import random
def ransac(matches1_2,bestcorners1,bestcorners2,iterations,threshold):
    matches12_shape=np.array(matches1_2).shape
    ans=[]
    for i in range(0,iterations):
        pts_src=[]
        pts_des=[]
        for j in range(0,4):
            num=random.randrange(len(matches1_2))
            dist,index1,index2=matches1_2[num]
            coord_1=bestcorners1[index1][0]##xy of img1
            coord_2=bestcorners2[index2][0]##xy of img2
            pts_src.append(np.array(coord_1))
            pts_des.append(np.array(coord_2))
        #print(pts_src)
        h, status = cv2.findHomography(np.array(pts_src), np.array(pts_des))
        inliers=[]
        pair=[]
        for i in range(len(matches1_2)):

            dist,index1,index2=matches1_2[i]
            coord_1=bestcorners1[index1][0]##xy of img1
            coord_2=bestcorners2[index2][0]##xy of img2
            coord1_dash=np.append(coord_1,1)
            #print(coord1_dash)
            coord1_2=np.matmul(h,coord1_dash)   #coord 1 in img2
            #print(coord1_2)
            if(coord1_2[2]==0):
                coord1_2[2]=0.00001
                
            coord1_2=coord1_2/coord1_2[2]
            ssd=(coord_2[1]-coord1_2[1])**2+(coord_2[0]-coord1_2[0])**2
            if ssd<threshold:
                inliers.append(i)

        ans.append(np.array([h,inliers,len(inliers)]))

    ans=np.array(ans)
    leninliers=ans[:,2]

    indices=np.argsort(leninliers)
    #print(indices[0])
    return ans,indices


In [3]:
img1=cv2.imread('1.jpg')
img2=cv2.imread('2.jpg')
img3=cv2.imread('3.jpg')

In [9]:
def ransac(pts1,pts2):             ################# can be torch
    best_inliers=[]
    best_F=[]
    
    for i in range(0,100):
        inliers=[]
        randomvals=[]
        co_ord=[]
        for j in range(0,9):
            randomvals.append(random.randint(0,len(pts1)-1))
            co_ord.append([pts1[randomvals[j]],pts2[randomvals[j]]])
       
        
        #print("coord",co_ord)
        A=[]
        for points in co_ord:
            x1,y1=points[0]
            x1dash,y1dash=points[1]
            row=[x1*x1dash,x1*y1dash,x1,y1*x1dash,y1*y1dash,y1,x1dash,y1dash,1]
            A.append(row)
        
        u,s,v=np.linalg.svd(A)
        
        F=v[-1]
        #print(F)
        F = np.reshape(F, (3, 3))
        F=F.T
      

    
        U, S, V = np.linalg.svd(F)
        S = np.array([[S[0], 0, 0], [0, S[1], 0], [0, 0, 0]])
        F = np.dot(U, S)
        F = np.dot(F, V)

        F = F / F[2, 2]
        
        for i in range(0,len(pts1)):
            p2=pts2[i]
            #print(p2)
            
            p2=np.array(p2)
            p2=np.append(p2,1)
            p2=p2.T
            
            
            p1=pts1[i]
            #print(p1)
            
            p1=np.append(p1,1)
            p1=p1.T
            #print(p1.shape,p2.shape)
            error=np.dot(np.dot(p2.T,F),p1)
            if abs(error)<0.01:
                #print("inlier")
                #print(error)
                inliers.append([pts1[i],pts2[i]])
            
        if len(inliers)>len(best_inliers):
            best_inliers=inliers
            best_F=F
                
        
            
    return best_inliers,best_F


def EssentialMatrixFromFundamentalMatrix(F, K):           ##### can be torch

    E = np.dot(K.T, np.dot(F, K))
    U, S, V_T = np.linalg.svd(E)

    E = np.dot(U, np.dot(np.diag([1, 1, 0]), V_T))
    return E
        

def ExtractCameraPose(E):                             ############can be torch
    W=np.array([[0,-1,0], [1,0,0] , [0,0,1] ])
    U,D,Vt=np.linalg.svd(E)
    C1=U[:,2]
    R1=U@W@Vt
    
    C2=-U[:,2]
    R2=U@W@Vt
    
    
    C3=U[:,2]
    R3=U@W.T@Vt
    
    C4=-U[:,2]
    R4=U@W.T@Vt
    
    if(np.linalg.det(R1)<-0.5):
        C1=-C1
        R1=-R1
        
    
    if(np.linalg.det(R2)<-0.5):
        C2=-C2
        R2=-R2
        
    
    if(np.linalg.det(R3)<-0.5):
        C3=-C3
        R3=-R3
        
    
    if(np.linalg.det(R4)<-0.5):
        C4=-C4
        R4=-R4
        
    return C1,R1,C2,R2,C3,R3,C4,R4
        

def skew(x):                               ################ can be torch
 
    return np.array([[0, -x[2], x[1]], [x[2], 0, x[0]], [x[1], x[0], 0]])
    
    
def LinearTriangulation(K, C1, R1, C2, R2, x1, x2):       ########## can be torch

    I = np.identity(3)
    print(x1.shape)
    
    sz = x1.shape[0]
    C1 = np.reshape(C1, (3, 1))
    C2 = np.reshape(C2, (3, 1))
    P1 = np.dot(K, np.dot(R1, np.hstack((I, -C1))))
    P2 = np.dot(K, np.dot(R2, np.hstack((I, -C2))))
    #print(P1)
    #print(P2)
    #     print(P2.shape)
    X1 = np.hstack((x1, np.ones((sz, 1))))
    X2 = np.hstack((x2, np.ones((sz, 1))))
    
    
    X = np.zeros((sz, 3))

    for i in range(sz):
        skew1 = skew(X1[i, :])
        skew2 = skew(X2[i, :])
        A = np.vstack((np.dot(skew1, P1), np.dot(skew2, P2)))
        _, _, v = np.linalg.svd(A)
        x = v[-1] / v[-1, -1]
        x = np.reshape(x, (len(x), -1))
        X[i, :] = x[0:3].T

    return X
    

    
def DisambiguateCameraPose(C1,R1,C2,R2,C3,R3,C4,R4,X1,X2,X3,X4):
    print()
    C=[]
    R=[]
    X=[]
    C.append(C1);C.append(C2);C.append(C3);C.append(C4);
    R.append(R1);R.append(R2);R.append(R3);R.append(R4);
    X.append(X1);X.append(X2);X.append(X3);X.append(X4);
    
    bestC=[]
    bestR=[]
    bestX=[]
    max=0
    for i in range(len(C)):
        counter=0
        C_=C[i]
        R_=R[i]
        X_=X[i]
        for x in X_:   #######Check how many pts in front
            if R_[2]@(x-C_) >0   and    x[2]>0:
                counter+=1
                
            
        if max<counter:
            max=counter
            bestR=R_
            bestC=C_
            bestX=X_
            
    return bestC,bestR,bestX
    
    
    
K = np.array([[568.996140852, 0, 643.21055941],
     [0, 568.988362396, 477.982801038],
     [0, 0, 1]])
sift = cv2.xfeatures2d.SIFT_create()
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

# FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params,search_params)

matches = flann.knnMatch(des1,des2,k=2)


# Need to draw only good matches, so create a mask


# ratio test as per Lowe's paper
good = []
pts1 = []
pts2 = []

# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
    if m.distance < 0.8*n.distance:
        good.append(m)
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)


pts1 = np.int32(pts1)
pts2 = np.int32(pts2)
inliers,F=ransac(pts1,pts2)
print("inliers ",len(inliers),inliers[0:3])
inliers=np.asarray(inliers)
pts1=np.array(inliers[:,0])
pts2=np.array(inliers[:,1])


print("check",len(pts1),pts1[0])
print("check1",len(pts2),pts2[0])

#F, mask1 = cv2.findFundamentalMat(pts1,pts2,cv2.FM_LMEDS)
# We select only inlier points
#pts1 = pts1[mask.ravel()==1]
#pts2 = pts2[mask.ravel()==1]

out=DrawCorrespondence(img1,img2,pts1,pts2)
#cv2.imshow("out",out)
#cv2.waitKey(0)
#cv2.destroyAllWindows()


E=EssentialMatrixFromFundamentalMatrix(F,K)
print(F,len(inliers),inliers[0],len(pts1))
print(E)
C1,R1,C2,R2,C3,R3,C4,R4=ExtractCameraPose(E)
X1=LinearTriangulation(K, np.zeros((3,1)), np.identity(3), C1, R1, pts1,pts2)
#####dont know why zeros and eye BCS FiRST CAMERA PLACED AT ORIGIN 
X2=LinearTriangulation(K, np.zeros((3,1)), np.identity(3), C2, R2, pts1,pts2)
X3=LinearTriangulation(K, np.zeros((3,1)), np.identity(3), C3, R3, pts1,pts2)
X4=LinearTriangulation(K, np.zeros((3,1)), np.identity(3), C4, R4, pts1,pts2)



C_,R_,X_=DisambiguateCameraPose(C1,R1,C2,R2,C3,R3,C4,R4,X1,X2,X3,X4)
#print(C_,R_,X_[0:2])
#NonLinearTriangulation(K,pts1,pts2,np.zeros((3,1)),np.identity(3),C_,R_,X_)
print(np.array(X_).shape)
print(np.array(pts1).shape)
print(np.array(pts2).shape)
#print(C_,R_,X_)
#print(ExtractCameraPose(E))

inliers  819 [[array([245, 169]), array([ 49, 250])], [array([246, 405]), array([ 16, 518])], [array([252, 425]), array([ 23, 545])]]
check 819 [245 169]
check1 819 [ 49 250]
[[-6.37598445e-07 -1.07934106e-05  2.48841516e-03]
 [ 1.26796996e-05 -1.02849934e-08 -6.48080067e-03]
 [-3.56137972e-03  4.09718599e-03  1.00000000e+00]] 819 [[245 169]
 [ 49 250]] 819
[[-0.0350504  -0.81797853 -0.40725707]
 [ 0.9382179  -0.0107756   0.21228444]
 [ 0.27731219 -0.38274335 -0.12245868]]
(819, 2)
(819, 2)
(819, 2)
(819, 2)

(819, 3)
(819, 2)
(819, 2)


In [8]:
def DrawCorrespondence(img1, img2, inliers_a, inliers_b):
    
    rows1, cols1 = img1.shape[:2]
    rows2, cols2 = img2.shape[:2]
    cv2.imshow("img1",img1)
    cv2.imshow("img2",img2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    out = np.zeros((max([rows1, rows2]), cols1 + cols2, 3), dtype='uint8')
    out[:rows1, :cols1, :] = img1
    out[:rows2, cols1:cols1 + cols2, :] = img2
    radius = 6
    BLUE = (255, 0, 0)
    GREEN = (0, 255, 0)
    RED = (0, 0, 255)
    thickness = 1

    for m in range(0, len(inliers_a)):
        # Draw small circle on image 1
        cv2.circle(out, (int(inliers_a[m][0]), int(inliers_a[m][1])),
            radius, RED, thickness)

        # Draw small circle on image 2
        cv2.circle(out, (int(inliers_b[m][0])
            + cols1, int(inliers_b[m][1])), radius, RED, thickness)

        # Draw line connecting circles
        cv2.line(out, (int(inliers_a[m][0]), int(inliers_a[m][1])),
            (int(inliers_b[m][0])+cols1, int(inliers_b[m][1])), GREEN, thickness)

    
    
    
    return out
    

In [10]:
import scipy
def NonLinearTriangulation(K,pts1,pts2,C1,R1,C2,R2,X_):
    print(X_.shape)
    C1=np.reshape(C1,(3,1))
    C2=np.reshape(C2,(3,1))
    R1=np.reshape(R1,(3,3))
    R2=np.reshape(R2,(3,3))
    print(X_[0])
    Xdash=X_.flatten()
    print(Xdash.shape)
    ans=scipy.optimize.least_squares(           ##############Change Xdash so that error decrs
        fun=myfunc,
        x0=Xdash,
        #method="dogbox",
        args=[K,pts1,pts2,C1,R1,C2,R2])
    
    X=np.reshape(ans.x,(int(len(ans.x)/3),3) )
    
    print(X[0],X.shape)
    return X
    
    
def myfunc(Xdash,K,pts1,pts2,C1,R1,C2,R2):
    I=np.identity(3)
    Xdash=np.reshape(Xdash,(int(len(Xdash)/3),3))
    #print(Xdash[0])
   
    noofpts=len(Xdash)
    #print(R1,C1,R2,C2,noofpts)
    P1 = np.dot(K, np.dot(R1, np.hstack((I, -C1))))
    P2 = np.dot(K, np.dot(R2, np.hstack((I, -C2))))
    
    Xdash=np.hstack((Xdash,np.ones((noofpts,1)) ))
    
    u1=P1[0,:]@Xdash.T
    v1=P1[1,:]@Xdash.T
    denom1=P1[2,:]@Xdash.T
    
    
    u2=P2[0,:]@Xdash.T
    v2=P2[1,:]@Xdash.T
    denom2=P2[2,:]@Xdash.T
    
    error11=0
    error12=0

    error21=0
    error22=0
    if len(pts1)>4:
        error11=pts1[:,0]-np.divide(u1,denom1)
        error12=pts1[:,1]-np.divide(v1,denom1)

        error21=pts2[:,0]-np.divide(u2,denom2)
        error22=pts2[:,1]-np.divide(v2,denom2)
    else:
        error11=pts1[0]-np.divide(u1,denom1)
        error12=pts1[1]-np.divide(v1,denom1)

        error21=pts2[0]-np.divide(u2,denom2)
        error22=pts2[1]-np.divide(v2,denom2)
        
    
    
    error1=error11+error12
    
    error2=error21+error22
    
    error=error1+error2
    #print(np.sum(error))
    
    return np.sum(error)
    
    
    
    
    
       
    
    
X_nLT=NonLinearTriangulation(K,pts1,pts2,np.zeros((3,1)),np.identity(3),C_,R_,X_)

 
    

(819, 3)
[-0.82559931 -0.37020143  0.58012293]
(2457,)
[0.01668139 0.3598822  1.98911453] (819, 3)


In [11]:
def LinearPNP(K,pts1,X_nLT):
    A=np.array([0,0,0])
    first_time=True
    for i in range(len(pts1)) :
        u,v=pts1[i]
        X=X_nLT[i]
        X=np.append(X,1)
        Xt=np.reshape(X,(1,4))
        first=np.array([
            [0,-1,v],
            [1 ,0, -u],
            [-v, u, 0]
        ])
        second_row1=np.hstack(( Xt,np.zeros((1,8)) ))
        second_row2=np.hstack(( np.zeros((1,4)), Xt))
        second_row2=np.hstack((second_row2, np.zeros((1,4)) ))
        second_row3=np.hstack((np.zeros((1,8)),Xt))
        second=np.vstack((second_row1,second_row2))
        second=np.vstack((second,second_row3))
        mat=first@second
        #print(mat.shape)
        if first_time==True:
            A=mat
            first_time=False
        else:
            A=np.vstack((A,mat))
    #print(A.shape)
    A=np.reshape(A,(-1,12))
    U,S,Vt=np.linalg.svd(A)
    ans=Vt[:,-1]
    P=np.reshape(ans,(3,4))
    return P[:,0:3], P[:,3]




    
    

In [21]:
def myfuncPNP(Rt,Xdash,K,pts1,C1,R1):
    I=np.identity(3)
    #Xdash=np.reshape(Xdash,(int(len(Xdash)/3),3))
    #print(Xdash[0])
    Rt=np.reshape(Rt,(3,4))
    R2=np.reshape(Rt[:,0:3],(3,3))
    C2=np.reshape(Rt[:,3],(3,1))
    
    noofpts=len(Xdash)
    #print(R1,C1,R2,C2,noofpts)
    P1 = np.dot(K, np.dot(R1, np.hstack((I, -C1))))
    
    Xdash=np.hstack((Xdash,np.ones((noofpts,1)) ))
    
    u1=P1[0,:]@Xdash.T
    v1=P1[1,:]@Xdash.T
    denom1=P1[2,:]@Xdash.T
    
    
    
    
    error11=0
    error12=0

    error21=0
    error22=0
    if len(pts1)>4:
        error11=pts1[:,0]-np.divide(u1,denom1)
        error12=pts1[:,1]-np.divide(v1,denom1)

        #error21=pts2[:,0]-np.divide(u2,denom2)
        #error22=pts2[:,1]-np.divide(v2,denom2)
    else:
        error11=pts1[0]-np.divide(u1,denom1)
        error12=pts1[1]-np.divide(v1,denom1)

        #error21=pts2[0]-np.divide(u2,denom2)
        #error22=pts2[1]-np.divide(v2,denom2)
        
    
    
    error1=error11+error12
    
    #error2=error21+error22
    
    #error=error1+error2
    #print(np.sum(error))
    
    return np.sum(error1)       
        
            
def myfuncPNPRansac(Xdash,K,pts1,C1,R1):
    I=np.identity(3)
    

    
    noofpts=len(Xdash)
    P1 = np.dot(K, np.dot(R1, np.hstack((I, -C1))))
    
    #Xdash=np.hstack((Xdash,np.ones((noofpts,1)) ))
    Xdash=np.append(Xdash,1)
    #Xdash=np.resize(Xdash,(1,4))
    
    
    u1=P1[0,:]@Xdash
    v1=P1[1,:]@Xdash
    denom1=P1[2,:]@Xdash
 
    error11=0
    error12=0

    error21=0
    error22=0
    if len(pts1)>4:
        error11=pts1[:,0]-np.divide(u1,denom1)
        error12=pts1[:,1]-np.divide(v1,denom1)

    else:
        error11=pts1[0]-np.divide(u1,denom1)
        error12=pts1[1]-np.divide(v1,denom1)

    
    error1=error11+error12

    #print(np.sum(error1))
    return np.sum(error1)       
        
            
        
    
                
    
        
def PnPRansac(K,pts1,X_nLT,threshold):
    errors=[]
    best_inliers=[]
    bestR=0
    bestC=0
    numofPoints=len(pts1)
    for i in range(0,1000):
        #print("PnpRansac")
        #print(len(pts1),len(pts2),len(X_nLT))
        
        randomnums=[]
        p1=[];p2=[];Xdash=[]
        for i in range(0,6):
            j=np.random.randint(0,numofPoints)
            p1.append(pts1[j])
            #p2.append(pts2[j])
            Xdash.append(X_nLT[j])
        R,C=LinearPNP(K,p1,Xdash)
        inliers=[]
        for j in range(numofPoints):
            error=myfuncPNPRansac(X_nLT[j],K,pts1[j],np.reshape(C,(3,1)),R)
            if abs(error)<threshold:
                inliers.append(j)
        
  
        
        if len(inliers)>len(best_inliers):
            best_inliers=inliers
            bestR=R
            bestC=C
    #print(bestC,len(best_inliers) )
    
    return bestR,bestC,best_inliers
    
            
                


        
    
        

    
    
    
#P=LinearPNP(K,pts1[0:12],X_nLT[0:12]) 
bestR,bestC,best_inliers=PnPRansac(K,pts1,X_nLT,3500)


In [25]:
def NonLinearPNP(K,pts1,C1,R1,X_):
    print(X_.shape)
    C1=np.reshape(C1,(3,1))
    #C2=np.reshape(C2,(3,1))
    R1=np.reshape(R1,(3,3))
    #R2=np.reshape(R2,(3,3))
    
    Rt=np.hstack((R1,C1))
    #print(Rt.shape)
    Rt=Rt.flatten()
    ans=scipy.optimize.least_squares(           ##############Change Xdash so that error decrs
        fun=myfuncPNP,
        x0=Rt,
        #method="dogbox",
        args=[X_,K,pts1,C1,R1])
    
    Rt=np.reshape(ans.x,(3,4) )
    
    print(Rt,Rt.shape)
    return Rt
    
    
print(C2,R2)
Rt=NonLinearPNP(K,pts1,C1,R1,X_)

[ 0.40475211  0.27306846 -0.87270232] [[ 0.97708017  0.05984295 -0.20428694]
 [-0.02594776  0.98599752  0.16472888]
 [ 0.21128428 -0.15565253  0.96495142]]
(819, 3)
[[ 0.97708017  0.05984295 -0.20428694 -0.40475211]
 [-0.02594776  0.98599752  0.16472888 -0.27306846]
 [ 0.21128428 -0.15565253  0.96495142  0.87270232]] (3, 4)


In [42]:
def GetInliersRansac(img1,img2):
    
    
    sift = cv2.xfeatures2d.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)

    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)

    flann = cv2.FlannBasedMatcher(index_params,search_params)

    matches = flann.knnMatch(des1,des2,k=2)

    good = []
    pts1 = []
    pts2 = []

    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.8*n.distance:
            good.append(m)
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)


    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)
    inliers,F=ransac(pts1,pts2)
    #print("inliers ",len(inliers),inliers[0:3])
    inliers=np.asarray(inliers)
    
    pts1=np.array(inliers[:,0])
    pts2=np.array(inliers[:,1])
    
    return (pts1[0:300],pts2[0:300],F)

    print("check",len(pts1),pts1[0])
    print("check1",len(pts2),pts2[0])



In [46]:
import copy
def make_pairs(imlist):
    nums=[i for i in range(0,len(imlist))]
    pairs1=[]
    for i in range(len(nums)):
        for j in range(i+1,len(nums)):
            pairs1.append((imlist[i],imlist[j]))
    
    return pairs1
    #print(pairs1)
    #pairs(0,nums)
    
    
def pairs(current,remaining):
    #print(current,remaining)
    #print()
    if len(remaining)==1:
        print(current)
        return
    for i in range(len(remaining)):
        rem=remaining[i]
        newlist=copy.copy(remaining)
        newlist.pop(i)
        pairs(current*10+rem,newlist)
        
##########################
#wrapper
##########################
img1=cv2.imread('1.jpg')
img2=cv2.imread('2.jpg')
img3=cv2.imread('3.jpg')
cv2.imshow('img1',img1)
cv2.destroyAllWindows()

K = np.array([[568.996140852, 0, 643.21055941],
     [0, 568.988362396, 477.982801038],
     [0, 0, 1]])


imlist=[]
imlist.append(img1)
imlist.append(img2)
imlist.append(img3);

nums=[i for i in range(0,len(imlist))]
#pairs=make_pairs(imlist)

pairsNum=[[0,i] for i in range(1,len(imlist))]
print(pairsNum)
pointsList=[]
for pair in pairsNum:
    print(pair[0],pair[1])
   
    p1,p2,F=GetInliersRansac(imlist[pair[0]],imlist[pair[1]])
    print(len(p1),len(p2))
    pointsList.append((p1,p2,F))


    
pts1,pts2,F=pointsList[0]
E=EssentialMatrixFromFundamentalMatrix(F,K)

C1,R1,C2,R2,C3,R3,C4,R4=ExtractCameraPose(E)
Cset=[];Cset.append(C1);Cset.append(C2);Cset.append(C3);Cset.append(C4);
Rset=[];Rset.append(R1);Rset.append(R2);Rset.append(R3);Rset.append(R4);
X1=LinearTriangulation(K, np.zeros((3,1)), np.identity(3), C1, R1, pts1,pts2)
#####dont know why zeros and eye BCS FiRST CAMERA PLACED AT ORIGIN 
X2=LinearTriangulation(K, np.zeros((3,1)), np.identity(3), C2, R2, pts1,pts2)
X3=LinearTriangulation(K, np.zeros((3,1)), np.identity(3), C3, R3, pts1,pts2)
X4=LinearTriangulation(K, np.zeros((3,1)), np.identity(3), C4, R4, pts1,pts2)



C_,R_,X_=DisambiguateCameraPose(C1,R1,C2,R2,C3,R3,C4,R4,X1,X2,X3,X4)
X_nLT=NonLinearTriangulation(K,pts1,pts2,np.zeros((3,1)),np.identity(3),C_,R_,X_)
Cset_=[C_]
Rset_=[R_]
for i in range(2,len(imlist)):
    Rnew,Cnew,_=PnPRansac(K,pts1,X_nLT,3500)
    Rt=NonLinearPNP(K,pts1,C1,R1,X_)
    Rnew=np.reshape(Rt[0:3,0:3],(3,3))
    Cnew=np.reshape(Rt[0:3,3],(3,1))
    
    Cset.append(Cnew);Rset.append(Rset);
    
    
    Xnew=LinearTriangulation(K,Cset_[0],Rset_[0],Cnew,Rnew,pts1,pointsList[i-1][1])
    X_nLTNew=NonLinearTriangulation(K,pts1,pointsList[i-1][1],Cset_[0],Rset_[0],Cnew,Rnew,X_nLT)
    
    

[[0, 1], [0, 2]]
0 1
300 300
0 2
300 300
(300, 2)
(300, 2)
(300, 2)
(300, 2)

(300, 3)
[-0.43050251 -0.01077172  0.26638054]
(900,)
[0.48908095 0.7778822  2.36464769] (300, 3)
(300, 3)
[[ 0.97622791  0.05072474 -0.21072749 -0.43669549]
 [-0.01377248  0.98478224  0.17324623 -0.16941486]
 [ 0.21630856 -0.16622556  0.96207051  0.88351325]] (3, 4)
(300, 2)
(300, 3)
[0.48908095 0.7778822  2.36464769]
(900,)
[0.32104632 0.4776048  2.66186903] (300, 3)
